In [1]:
from pyspark.sql import SparkSession

In [2]:
# 🧠 Étape 2 : Créer la session Spark avec les drivers JDBC
spark = SparkSession.builder \
    .appName("Test MySQL Ingestion") \
    .config("spark.driver.extraClassPath", "/usr/local/spark/jars/*") \
    .getOrCreate()

In [4]:

# ✅ Étape 3 : Lire la table MySQL
df_mysql = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://mysql:3306/hospital_legacy") \
    .option("dbtable", "patients_legacy") \
    .option("user", "source_reader") \
    .option("password", "Source_Read_2024!") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .load()

In [8]:
df_mysql.show(50)

+---+------------+----------+---------+-------------+------+--------------------+----------+----------+-------------------+------------+-----------------+-------------------+
| id|patient_code|first_name|last_name|date_of_birth|gender|             address|      city|     phone|              email|insurance_id|registration_date|       last_updated|
+---+------------+----------+---------+-------------+------+--------------------+----------+----------+-------------------+------------+-----------------+-------------------+
|  1|      LEG001|     Ahmed|   Benali|   1980-03-15| M    |   12 Rue de la Paix|Casablanca|0611223344|  ahmed.b@email.com|   INS-45678|       2020-01-10|2026-02-23 11:19:44|
|  2|      LEG002|    Fatima|    Zahra|   1975-07-22| F    | 45 Avenue Hassan II|     Rabat|0622334455| fatima.z@email.com|   INS-56789|       2019-05-15|2026-02-23 11:19:44|
|  3|      LEG003|      Omar|  Idrissi|   1990-11-08| M    |78 Boulevard Moha...| Marrakech|0633445566|   omar.i@email.com|  

In [10]:
# Lire le fichier CSV
df_csv = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/home/jovyan/work/data/patients_source.csv")

# Afficher les premières lignes
df_csv.show(10)


+----------+----------+---------+-------------+------+-----------+----------+---------+--------------------+-----------------+
|patient_id|first_name|last_name|date_of_birth|gender|       city|blood_type|    phone|               email|registration_date|
+----------+----------+---------+-------------+------+-----------+----------+---------+--------------------+-----------------+
|   PAT-001|      Jean|   Martin|   1985-03-15|     M|      Paris|        A+|612345678|jean.martin@email...|       2024-01-10|
|   PAT-002|     Marie|  Bernard|   1990-07-22|     F|       Lyon|        O-|698765432|marie.bernard@ema...|       2024-01-15|
|   PAT-003|    Pierre|   Dubois|   1978-11-08|     M|  Marseille|        B+|654321987|pierre.dubois@ema...|       2024-02-01|
|   PAT-004|    Sophie|   Thomas|   1982-09-30|     F|   Toulouse|       AB+|678912345|sophie.thomas@ema...|       2024-02-20|
|   PAT-005|    Michel|   Robert|   1975-12-12|     M|       Nice|        A-|632165498|michel.robert@ema...|   

In [12]:
import requests
import pandas as pd

# URL de l'API (depuis le conteneur Spark, utilise le nom du service Docker)
url = "http://api-medical:8000/visits"  # ← adapte ce endpoint selon ton API

# Requête GET
response = requests.get(url)

# Vérifie la réponse
if response.status_code == 200:
    data = response.json()
    df_api = pd.DataFrame(data)
    display(df_api)
else:
    print(f"Erreur {response.status_code} : {response.text}")


,count,visits
0,48,{'visit_id': '7c8037be-3c2d-4c8f-8159-77c76166...
1,48,{'visit_id': '12e69204-4665-4413-b082-5948a7fc...
2,48,{'visit_id': 'b2c862a2-da77-4c30-b920-cdd9e491...
3,48,{'visit_id': '6eea0e70-7430-414c-9da8-032fdd3a...
4,48,{'visit_id': 'b9e4b181-a6c6-44e1-8e5d-c44bf440...
5,48,{'visit_id': '569dd8d4-d1c5-41cf-82cb-9d29955b...
6,48,{'visit_id': 'de450b2a-0683-4f03-9bf7-0590678d...
7,48,{'visit_id': 'dd6892b3-8cd0-4056-827c-b68d84f2...
8,48,{'visit_id': 'f0f6f059-05f5-4be5-9af0-a96066bf...
9,48,{'visit_id': '9a5f3e56-9478-4c17-bf74-a0585892...
